In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import scipy
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

/Users/taoshen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/taoshen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/taoshen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/taoshen/opt/anaconda3/lib/python3.7/site-packages/tensorf

# FM
$$
\hat y(x) = w_0+\sum_{i=1}^n w_i x_i +\sum_{i=1}^n \sum_{j=i+1}^n ⟨vi,vj⟩ x_i x_j \\
=w_0+\sum_{i=1}^n w_i x_i + \frac{1}{2} \sum_{f=1}^{k} {\left \lgroup \left(\sum_{i=1}^{n} v_{i,f} x_i \right)^2 - \sum_{i=1}^{n} v_{i,f}^2 x_i^2\right \rgroup} \qquad
$$

In [2]:
from itertools import count
import collections
def vectorize_dic(dic,label2index=None,hold_num=None):
    if not label2index:
        d = count(0)
        label2index = collections.defaultdict(lambda:next(d))
    if not hold_num:
        hold_num = len(label2index)
    sample_num = len(list(dic.values())[0])
    feature_num = len(dic.values())
    total_num = sample_num * feature_num
    
    col_idx = np.empty(total_num,dtype=int)  # 列索引
    
    # i=0对所有的user做映射，i=1对所有的item做映射
    i = 0
    for feat,lis in dic.items():
        col_idx[i::feature_num] = [label2index[str(feat)+str(el)] for el in lis]
        i+=1
    
    row_idx = np.repeat(np.arange(sample_num),feature_num)
    data = np.ones(total_num)
    
    left_data_index = np.where(col_idx<hold_num)
    
    return scipy.sparse.csr.csr_matrix((data[left_data_index],(row_idx[left_data_index],col_idx[left_data_index])),
                                       shape=(sample_num,feature_num)),label2index
    
    

def load_data():
    ratings_path = '../data/ml-1m/ratings.dat'
    ratingsDF = pd.read_csv(ratings_path,index_col=None,sep='::',header=None,names=['user','item','rating','timestamp'],nrows=1000)
    
    ratingsDF = ratingsDF.sample(frac=1.0)
    cut_idx = int(ratingsDF.shape[0]*0.7)
    train,test = ratingsDF.iloc[:cut_idx],ratingsDF.iloc[cut_idx:]
    
    x_train,label2index = vectorize_dic({'users':train['user'].values,'item':train['item'].values})
    x_test,_ = vectorize_dic({'users':test['user'].values,'item':test['item'].values},label2index,x_train.shape[1])
    
    x_train,x_test = x_train.todense(),x_test.todense()
    y_train,y_test = train['rating'].values,test['rating'].values
    
    return x_train,x_test,y_train,y_test
    
def batcher(x,y,batch_size=-1):
    sample_num = x.shape[0]
    if batch_size == -1:
        batch_size = sample_num
    for i in range(0,sample_num,batch_size):
        upper_bound = min(i+batch_size,sample_num)
        batch_x = x[i:upper_bound]
        batch_y = y[i:upper_bound]
        yield (batch_x,batch_y)

        
# 所有feat（onehot之后）映射到一个vec_embedding
    
def main():
    x_train,x_test,y_train,y_test = load_data()
    
    vec_dim = 10
    
    sample_num,feat_num = x_train.shape
    batch_size = 32
    epochs = 20
    learning_rate = 0.01
    
    x = tf.placeholder(tf.float32,shape=[None,feat_num],name='x')
    y = tf.placeholder(tf.float32,shape=[None,1],name='y')
    
    w0 = tf.get_variable(name='bias',shape=(1),dtype=tf.float32)
    W = tf.get_variable(name='linear_w',shape=(feat_num,1),dtype=tf.float32)
    #这里也可以初始化为shape=(feat_num),后面使用multiply，效果一致
    V = tf.get_variable(name='interaction_w',shape=(feat_num,vec_dim),dtype=tf.float32)
    
    linear_part = w0 + tf.reduce_sum(tf.matmul(x,W),axis=1,keep_dims=True)
    interaction_part = 0.5 * tf.reduce_sum(tf.square(tf.matmul(x,V))-tf.matmul(tf.square(x),tf.square(V)),axis=1,keep_dims=True)
    y_hat = linear_part + interaction_part
    loss = tf.reduce_mean(tf.square(y-y_hat))
    
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    init = tf.global_variables_initializer()
    
    
    with tf.Session() as sess:
        sess.run(init)
        for e in range(epochs):
            print('Epoch:%i'%e)
            step = 0
            for batch_x,batch_y in batcher(x_train,y_train,batch_size=batch_size):
                sess.run(train_op,feed_dict={x:batch_x,y:batch_y.reshape(-1,1)})
                step += 1
                if step % 10 == 0:
                    test_loss = sess.run(loss,feed_dict={x:x_test,y:y_test.reshape(-1,1)})
                    print('Step:%d,test MSE:%.3f'%(step,test_loss))
                    
tf.reset_default_graph()     
main()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch:0
Step:10,test MSE:6.162
Step:20,test MSE:5.734
Epoch:1
Step:10,test MSE:5.256
Step:20,test MSE:4.883
Epoch:2
Step:10,test MSE:4.473
Step:20,test MSE:4.156
Epoch:3
Step:10,test MSE:3.809
Step:20,test MSE:3.542
Epoch:4
Step:10,test MSE:3.253
Step:20,test MSE:3.032
Epoch:5
Step:10,test MSE:2.794
Step:20,test MSE:2.614
Epoch:6
Step:10,test MSE:2.421
Step:20,test MSE:2.276
Epoch:7
Step:10,test MSE:2.122
Step:20,test MSE:2.007
Epoch:8
Step:10,test MSE:1.887
Step:20,test MSE:1.798
Epoch:9
Step:10,test MSE:1.707
Step:20,test MSE:1.639
Epoch:10
Step:10,test MSE:1.571
Step:20,test MSE:1.521
Epoch:11
Step:10,test MSE:1.471
Step:20,test MSE:1.436
Epoch:12
Step:10,test MSE:1.402
Step:20,test MSE:1.377
Epoch:13
Step:10,test MSE:1.354
Step:20,test MSE:1.339
Epoch:14
Step:10,test MSE:1.325
Step:20,test MSE:

In [3]:

train_path = '../data/ctr/train.csv'
test_path = '../data/ctr/test.csv'

dfTrain = pd.read_csv(train_path)
dfTest = pd.read_csv(test_path)

def preprocess(df):
    cols = [c for c in df.columns if c not in ['id','target']]
    df["missing_feat"] = np.sum((df[cols] == -1).values, axis=1)
    df['ps_car_13_x_ps_reg_03'] = df['ps_car_13'] * df['ps_reg_03']
    return df
    

dfTrain,dfTest = preprocess(dfTrain),preprocess(dfTest)
IGNORE_COLS = [
    "id", "target",
    "ps_calc_01", "ps_calc_02", "ps_calc_03", "ps_calc_04",
    "ps_calc_05", "ps_calc_06", "ps_calc_07", "ps_calc_08",
    "ps_calc_09", "ps_calc_10", "ps_calc_11", "ps_calc_12",
    "ps_calc_13", "ps_calc_14",
    "ps_calc_15_bin", "ps_calc_16_bin", "ps_calc_17_bin",
    "ps_calc_18_bin", "ps_calc_19_bin", "ps_calc_20_bin"
]
NUMERIC_COLS = [
    "ps_reg_01", "ps_reg_02", "ps_reg_03",
    "ps_car_12", "ps_car_13", "ps_car_14", "ps_car_15"
]
cols = [c for c in dfTrain.columns if (not c in IGNORE_COLS)]

x_train,x_test = dfTrain[cols].values,dfTest[cols].values
y_train = dfTrain['target'].values
ids_test = dfTest['id'].values


cat_features_indices = [i for i,c in enumerate(cols)]

# DeepFM

In [4]:
#DeepFM数据预处理
dfAll = pd.concat([dfTrain,dfTest])

feature_dic = {}
total_feature = 0
for col in dfAll:
    if col in IGNORE_COLS:
        continue
    elif col in NUMERIC_COLS:
        feature_dic[col] = total_feature
        total_feature += 1
    else:
        unique_val = dfAll[col].unique()
        feature_dic[col] = dict(zip(unique_val,range(total_feature,total_feature+len(unique_val))))
        total_feature += len(unique_val)
        
def process(df,feature_dic):
    dfi = df.copy()
    dfv = df.copy()
    for col in df.columns:
        if col in IGNORE_COLS:
            dfi.drop(col,axis=1,inplace=True)
            dfv.drop(col,axis=1,inplace=True)
        elif col in NUMERIC_COLS:
            dfi[col] = feature_dic[col]
        else:
            dfi[col] = dfi[col].map(feature_dic[col])
            dfv[col] = 1
    return dfi,dfv
        
        
train_feature_index,train_feature_value = process(dfTrain,feature_dic)   
test_feature_index,test_feature_value = process(dfTest,feature_dic)   


dfm_params = {
    'use_fm':True,
    'use_deep':True,
    'embedding_size':8,
    'dropout_fm':[1,1],
    'deep_layers':[32,32],
    'dropout_deep':[0.5,0.5,0.5],
    'deep_layer_activation':tf.nn.relu,
    'epoch':30,
    'learning_rate':0.01,
    'batch_size':64,
    'optimizer':'adam',
    'batch_norm':1,
    'batch_decay':0.95,
    'l2_reg':0.01,
    'verbose':True,
    'eval_metric':'gini_norm',
    'random_seed':3
}

dfm_params['feature_size'] = total_feature
dfm_params['field_size'] = train_feature_index.shape[1]

# 构建图
tf.reset_default_graph()  


feat_index = tf.placeholder(tf.int32,shape=[None,None],name='index')
feat_value = tf.placeholder(tf.float32,shape=[None,None],name='value')
label = tf.placeholder(tf.float32,shape=[None,1],name='label')

weights = {}
weights['feature_embeddings'] = tf.Variable(tf.random_normal([dfm_params['feature_size'],dfm_params['embedding_size']],0.,0.01),name='feature_embeddings')
weights['feature_bias'] = tf.Variable(tf.random_normal([dfm_params['feature_size'],1],0.,0.1),name='feature_bias')


num_layers = len(dfm_params['deep_layers'])
input_size = dfm_params['field_size']*dfm_params['embedding_size']  # 每一个field做embedding
glorot = np.sqrt(2./(input_size+dfm_params['deep_layers'][0]))
weights['layer_0'] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=(input_size,dfm_params['deep_layers'][0])),dtype=np.float32)
'''weights中bias取一个维度即可，且输入层不需要bias'''
weights['bias_0'] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=(dfm_params['deep_layers'][0])),dtype=np.float32)

for i in range(1,num_layers):
    glorot = np.sqrt(2./(dfm_params['deep_layers'][i-1]+dfm_params['deep_layers'][i]))
    weights['layer_'+str(i)] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=(dfm_params['deep_layers'][i-1],dfm_params['deep_layers'][i])),dtype=np.float32)
    weights['bias_'+str(i)] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=(dfm_params['deep_layers'][i])),dtype=np.float32)

'''最后一层为FM层+DNN最后一层'''
input_size = dfm_params['deep_layers'][-1]+dfm_params['field_size']+dfm_params['embedding_size']
glorot = np.sqrt(2./(input_size+1))

weights['concat_project'] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=(input_size,1)),dtype=np.float32)
weights['concat_bias'] = tf.Variable(tf.constant(0.01),dtype=np.float32)




'''embedding'''
embeddings = tf.nn.embedding_lookup(weights['feature_embeddings'],feat_index) 
reshaped_feat_value = tf.reshape(feat_value,[-1,dfm_params['field_size'],1])
embeddings = tf.multiply(embeddings,reshaped_feat_value)

'''fm part'''
'''一阶'''
fm_first_order = tf.nn.embedding_lookup(weights['feature_embeddings'],feat_index)
fm_first_order = tf.reduce_sum(tf.multiply(fm_first_order,reshaped_feat_value),2)

summed_feature_emb = tf.reduce_sum(embeddings,1)
summed_feature_emb_square = tf.square(summed_feature_emb)

squared_feature_emb = tf.square(embeddings)
squared_sum_feature_emb = tf.reduce_sum(squared_feature_emb,1)

fm_second_order = 0.5 * tf.subtract(summed_feature_emb_square,squared_sum_feature_emb)

'''deep part'''
y_deep = tf.reshape(embeddings,shape=[-1,dfm_params['field_size']*dfm_params['embedding_size']])

for i in range(num_layers):
    y_deep = tf.add(tf.matmul(y_deep,weights['layer_'+str(i)]),weights['bias_'+str(i)])
    y_deep = tf.nn.relu(y_deep)

concat_input = tf.concat([fm_first_order,fm_second_order,y_deep],axis=1)
out = tf.nn.sigmoid(tf.add(tf.matmul(concat_input,weights['concat_project']),weights['concat_bias']))


loss = tf.losses.log_loss(label,out)
train_op = tf.train.AdamOptimizer(learning_rate = dfm_params['learning_rate'],beta1=0.9,beta2=0.999,epsilon=1e-8).minimize(loss)


init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(dfm_params['epoch']):
        '''loss与train_op顺序不可换'''
        epoch_loss,_ = sess.run([loss,train_op],feed_dict={feat_index:train_feature_index,feat_value:train_feature_value,label:y_train.reshape(-1,1)})
        print('epoch:%s,Loss:%s'%(i,epoch_loss))
        

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
epoch:0,Loss:0.7773253
epoch:1,Loss:0.5417052
epoch:2,Loss:0.35673484
epoch:3,Loss:0.22753826
epoch:4,Loss:0.16803075
epoch:5,Loss:0.15897305
epoch:6,Loss:0.17649129
epoch:7,Loss:0.19727606
epoch:8,Loss:0.20989627
epoch:9,Loss:0.21218628
epoch:10,Loss:0.2060652
epoch:11,Loss:0.19401805
epoch:12,Loss:0.17890438
epoch:13,Loss:0.16328596
epoch:14,Loss:0.14911337
epoch:15,Loss:0.13857953
epoch:16,Loss:0.13112995
epoch:17,Loss:0.1255495
epoch:18,Loss:0.12259299
epoch:19,Loss:0.11945083
epoch:20,Loss:0.11505127
epoch:21,Loss:0.10915083
epoch:22,Loss:0.100132756
epoch:23,Loss:0.08852969
epoch:24,Loss:0.076438524
epoch:25,Loss:0.06531809
epoch:26,Loss:0.056068007
epoch:27,Loss:0.046559855
epoch:28,Loss:0.036692847
epoch:29,Loss:0.027390907


# DCN
$$x_{l+1} = x_0 x_l^T w_l + b_l + x_l = f(x_l,w_l,b_l)+x_l$$

In [5]:
#DCN处理
dfAll = pd.concat([dfTrain,dfTest])

feature_dic = {}
total_feature = 0
for col in dfAll:
    if col in IGNORE_COLS or col in NUMERIC_COLS:
        continue
    else:
        unique_val = dfAll[col].unique()
        feature_dic[col] = dict(zip(unique_val,range(total_feature,total_feature+len(unique_val))))
        total_feature += len(unique_val)
        
'''
主要将离散特征和连续特征分开，连续特征不在转换成embedding进行输入
'''     
def process(df,feature_dic):
    num_val = df[NUMERIC_COLS].values  #.tolist()
    '''只保留类别特征'''
    dfi = df.copy()
    dfi.drop(NUMERIC_COLS,axis=1,inplace=True)
    dfv = dfi.copy()
    for col in dfi.columns:
        if col in IGNORE_COLS:
            dfi.drop(col,axis=1,inplace=True)
            dfv.drop(col,axis=1,inplace=True)
        else:
            dfi[col] = dfi[col].map(feature_dic[col])
            dfv[col] = 1
    dfi = dfi.values
    dfv = dfv.values
    
    return dfi,dfv,num_val
    
train_cate_feature_index,train_cate_feature_value,train_num_feature_val = process(dfTrain,feature_dic)   
test_cate_feature_index,test_feature_cate_value,test_num_feature_val = process(dfTest,feature_dic)   
      

dcn_params = {
    "embedding_size": 8,
    "deep_layers": [32, 32],
    "dropout_deep": [0.5, 0.5, 0.5],
    "deep_layers_activation": tf.nn.relu,
    'loss' : 'logloss',
    "epoch": 30,
    "batch_size": 1024,
    "learning_rate": 0.001,
    "optimizer_type": "adam",
    "batch_norm": 1,
    "batch_norm_decay": 0.995,
    "l2_reg": 0.01,
    "verbose": True,
    "random_seed": 2019,
    "cross_layer_num":3
}

dcn_params['cate_feature_size'] = total_feature
dcn_params['field_size'] = train_cate_feature_index.shape[1]
dcn_params['numeric_feature_size'] = train_num_feature_val.shape[1]
total_size = dcn_params['field_size']*dcn_params['embedding_size']+dcn_params['numeric_feature_size']  # 每一个field做embedding

# 构建图
tf.reset_default_graph()
tf.set_random_seed(dcn_params['random_seed'])

feat_index = tf.placeholder(tf.int32,shape=[None,None],name='feat_index')
feat_value = tf.placeholder(tf.float32,shape=[None,None],name='feat_value')
numeric_value = tf.placeholder(tf.float32,shape=[None,None],name='numeric_val')
label = tf.placeholder(tf.float32,shape=[None,1],name='label')


weights = {}
weights['feature_embedding'] = tf.Variable(tf.random_normal(shape=[dcn_params['cate_feature_size'],dcn_params['embedding_size']],mean=0.,stddev=0.01),name='feature_embeddings')
weights['feature_bias'] = tf.Variable(tf.random_normal(shape=[dcn_params['cate_feature_size']],mean=0,stddev=1),name='feature_bias')




'''deep'''
input_size = total_size
glorot = np.sqrt(2./(input_size+dcn_params['deep_layers'][0]))
weights['deep_layers_0'] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=[input_size,dcn_params['deep_layers'][0]]),dtype=np.float32,name='layers_0')
weights['deep_bias_0'] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=[dcn_params['deep_layers'][0]]),dtype=np.float32,name='bias_0')


num_layers = len(dcn_params['deep_layers'])
for i in range(1,num_layers):
    glorot = np.sqrt(2./(dcn_params['deep_layers'][i-1]+dcn_params['deep_layers'][i]))
    weights['deep_layers_'+str(i)] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=[dcn_params['deep_layers'][i-1],dcn_params['deep_layers'][i]]),dtype=np.float32,name='deep_layers_'+str(i))
    weights['deep_bias_'+str(i)] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=[dcn_params['deep_layers'][i]]),dtype=np.float32,name='deep_bias_'+str(i))


# glorot = np.sqrt(2./(input_size*2))    # 初始化
for i in range(dcn_params['cross_layer_num']):
    weights['cross_layers_'+str(i)] = tf.Variable(np.random.normal(loc=0,scale=0.01,size=[total_size,1]),dtype=np.float32,name='cross_layers_'+str(i))
    weights['cross_bias_'+str(i)] = tf.Variable(np.random.normal(loc=0,scale=0.01,size=[total_size,1]),dtype=np.float32,name='cross_bias_'+str(i))

input_size += dcn_params['deep_layers'][-1]     
glorot = np.sqrt(2./(input_size+1))
weights['concat_projection'] = tf.Variable(np.random.normal(loc=0,scale=glorot,size=[input_size,1]),dtype=np.float32,name='concat_projection')
weights['concat_bias'] = tf.Variable(tf.constant(0.01),dtype=np.float32)                                                        
                                                                                             
                                                                                             
embeddings = tf.nn.embedding_lookup(weights['feature_embedding'],feat_index)
reshaped_feat_value = tf.reshape(feat_value,[-1,dcn_params['field_size'],1])
embeddings = tf.multiply(embeddings,reshaped_feat_value)

x0 = tf.concat([numeric_value,tf.reshape(embeddings,[-1,dcn_params['field_size']*dcn_params['embedding_size']])],axis=1)

'''deep part'''
y_deep = tf.nn.dropout(x0,dcn_params['dropout_deep'][0])
for i in range(num_layers):
    y_deep = tf.add(tf.matmul(y_deep,weights['deep_layers_'+str(i)]),weights['deep_bias_'+str(i)])
    y_deep = tf.nn.relu(y_deep)
    y_deep = tf.nn.dropout(y_deep,dcn_params['dropout_deep'][i+1])
    
'''cross part'''
_x0 = tf.reshape(x0,[-1,total_size,1])
x_l = _x0
for l in range(dcn_params['cross_layer_num']):
#     x_l = tf.tensordot(tf.matmul(_x0, x_l, transpose_b=True),
#                                     weights["cross_layers_%d" % l],1) + weights["cross_bias_%d" % l] + x_l
    # 注意计算顺序，可以加速很多
    x_l = tf.tensordot(tf.reshape(x_l,[-1,1,total_size]),weights["cross_layers_%d" % l],1) * _x0 + weights["cross_bias_%d" % l] + x_l


cross_network_out = tf.reshape(x_l,[-1,total_size])



concat_input = tf.concat([y_deep,cross_network_out],axis=1)
out = tf.add(tf.matmul(concat_input,weights['concat_projection']),weights['concat_bias'])




if dcn_params['loss'] == 'logloss':
    out = tf.nn.sigmoid(out)
    loss = tf.losses.log_loss(label,out)
elif dcn_params['loss'] == 'mse':
    loss = tf.nn.l2_loss(tf.subtract(label,out))

if dcn_params['l2_reg']>0:
    loss += tf.contrib.layers.l2_regularizer(dcn_params['l2_reg'])(weights['concat_projection'])
    for i in range(num_layers):
        loss += tf.contrib.layers.l2_regularizer(dcn_params['l2_reg'])(weights['deep_layers_%d'%i])
    for i in range(dcn_params['cross_layer_num']):
        loss += tf.contrib.layers.l2_regularizer(dcn_params['l2_reg'])(weights['cross_layers_%d'%i])
if dcn_params['optimizer_type'] == 'adam':
    train_op = tf.train.AdamOptimizer(learning_rate = dcn_params['learning_rate'],beta1=0.9,beta2=0.99,epsilon=1e-8).minimize(loss)
elif dcn_params['optimizer_type'] == 'adagrad':
    train_op = tf.train.AdagradOptimizer(learning_rate = dcn_params['learning_rate'],initial_accumulator_value=1e-8).minimize(loss)
elif dcn_params['optimizer_type'] == 'gd':
    train_op = tf.train.GradientDescentOptimizer(learning_rate=dcn_params['learning_rate']).minimize(loss)
elif dcn_params['optimizer_type'] == 'momentum':
    train_op = tf.train.MomentumOptimizer(learning_rate = dcn_params['learning_rate'],momentum=0.95).minimize(loss)
        
init = tf.global_variables_initializer()


def shuffle_in_union_scary(a,b,c,d):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)
    np.random.set_state(rng_state)
    np.random.shuffle(c)
    np.random.set_state(rng_state)
    np.random.shuffle(d)

def getbatch(Xi,Xv,Xv2,y,batch_size,index):
    start = batch_size * index
    end = min(batch_size * (index + 1),len(y))
    return Xi[start:end],Xv[start:end],Xv2[start:end],y[start:end]
  

with tf.Session() as sess:
    sess.run(init)
    for e in range(dcn_params['epoch']):
        shuffle_in_union_scary(train_cate_feature_index,train_cate_feature_value,train_num_feature_val,y_train)
        for i in range(len(y_train)//dcn_params['batch_size']):
            cate_Xi_batch,cate_Xv_batch,numeric_Xv_batch,y_batch = getbatch(train_cate_feature_index,train_cate_feature_value,train_num_feature_val,y_train,dcn_params['batch_size'],e)
            '''注意取名，不能与loss重复'''
            cur_loss,_ = sess.run([loss,train_op],feed_dict={feat_index:cate_Xi_batch,feat_value:cate_Xv_batch,numeric_value:numeric_Xv_batch,label:y_batch.reshape(-1,1)})
        print('Epoch%i : Loss %.2f'%(e,cur_loss))
            
            
            
            
            

    


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Epoch0 : Loss 0.97
Epoch1 : Loss 0.69
Epoch2 : Loss 0.53
Epoch3 : Loss 0.45
Epoch4 : Loss 0.41
Epoch5 : Loss 0.38
Epoch6 : Loss 0.37
Epoch7 : Loss 0.33
Epoch8 : Loss 0.29
Epoch9 : Loss 0.32
Epoch10 : Loss 0.13
Epoch11 : Loss 0.12
Epoch12 : Loss 0.11
Epoch13 : Loss 0.10
Epoch14 : Loss 0.09
Epoch15 : Loss 0.08
Epoch16 : Loss 0.07
Epoch17 : Loss 0.06
Epoch18 : Loss 0.06
Epoch19 : Loss 0.05
Epoch20 : Loss 0.05
Epoch21 : Loss 0.04
Epoch22 : Loss 0.04
Epoch23 : Loss 0.04
Epoch24 : Loss 0.03
Epoch25 : Loss 0.03
Epoch26 : L